In [1]:
import cv2
import numpy as np
import logging
import os
from collections import deque
import subprocess


def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "processing.log")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file),  # Salva log su file
            logging.StreamHandler()        # Mostra log su console
        ]
    )
    logging.info(f"Logging configurato. File di log salvato in: {log_file}")


def add_transparent_overlay(frame, boxes, color=(0, 0, 255), alpha=0.3):
    overlay = frame.copy()
    for (x, y, w, h) in boxes:
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)


def temporal_smoothing_flow(
    video_path, 
    output_dir, 
    flow_threshold=0.5,
    alpha_fraction=0.2,
    window_size=30,
    morph_kernel=3,
    save_name="overlay.mp4",
    mask_save_name="mask.mp4",
    margin=10,
    scale_factor=0.5
):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    overlay_path = os.path.join(output_dir, save_name)
    mask_path = os.path.join(output_dir, mask_save_name)

    out_overlay = cv2.VideoWriter(overlay_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height))
    out_mask = cv2.VideoWriter(mask_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height), isColor=False)

    ret, first_frame = cap.read()
    prev_gray_full = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.resize(prev_gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

    mask_queue = deque(maxlen=window_size)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_kernel, morph_kernel))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_full = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 2, 9, 2, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=False)
        mask_current_resized = (mag > flow_threshold).astype(np.uint8) * 255
        mask_current = cv2.resize(mask_current_resized, (width, height), interpolation=cv2.INTER_NEAREST)

        mask_queue.append(mask_current)
        cumulative_mask = np.sum(np.array(mask_queue), axis=0)
        mask_smoothed = (cumulative_mask >= (alpha_fraction * len(mask_queue) * 255)).astype(np.uint8) * 255
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_CLOSE, kernel)
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(mask_smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        boxes = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x_padded = max(0, x - margin)
            y_padded = max(0, y - margin)
            w_padded = min(w + 2 * margin, width - x_padded)
            h_padded = min(h + 2 * margin, height - y_padded)
            boxes.append((x_padded, y_padded, w_padded, h_padded))

        if boxes:
            add_transparent_overlay(frame, boxes)

        mask_rect = np.zeros((height, width), dtype=np.uint8)
        for (x, y, w, h) in boxes:
            cv2.rectangle(mask_rect, (x, y), (x + w, y + h), 255, -1)

        out_overlay.write(frame)
        out_mask.write(mask_rect)

        prev_gray = gray.copy()

    cap.release()
    out_overlay.release()
    out_mask.release()


def compress_with_ffmpeg(input_path, output_path):
    ffmpeg_command = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libx265",
        "-crf", "28",
        "-preset", "ultrafast",
        "-y",
        output_path
    ]
    try:
        subprocess.run(ffmpeg_command, check=True)
        logging.info(f"Video compresso salvato in: {output_path}")
    except subprocess.CalledProcessError as e:
        logging.error(f"Errore nella compressione con ffmpeg: {e}")


def compress_with_motion(input_video, mask_video, output_dir):
    cap_input = cv2.VideoCapture(input_video)
    cap_mask = cv2.VideoCapture(mask_video)

    fps = cap_input.get(cv2.CAP_PROP_FPS)
    width = int(cap_input.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_input.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_video = os.path.join(output_dir, "compressed.mp4")
    out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height))

    while True:
        ret_in, frame_in = cap_input.read()
        ret_mask, frame_mask = cap_mask.read()
        if not ret_in or not ret_mask:
            break

        if len(frame_mask.shape) == 3:
            frame_mask = cv2.cvtColor(frame_mask, cv2.COLOR_BGR2GRAY)

        static_mask = frame_mask == 0
        frame_in[static_mask] = (frame_in[static_mask] * 0.2).astype(np.uint8)

        out.write(frame_in)

    cap_input.release()
    cap_mask.release()
    out.release()

    optimized_video = os.path.join(output_dir, "compressed_optimized.mp4")
    compress_with_ffmpeg(output_video, optimized_video)


def main():
    input_dir = "../Dataset/input/"
    output_dir = "../Dataset/output/"
    input_video = [f for f in os.listdir(input_dir) if f.endswith(".mp4")][0]
    video_name = os.path.splitext(input_video)[0]
    video_output_dir = os.path.join(output_dir, video_name)

    if not os.path.exists(video_output_dir):
        os.makedirs(video_output_dir)

    setup_logging(video_output_dir)

    temporal_smoothing_flow(
        video_path=os.path.join(input_dir, input_video),
        output_dir=video_output_dir,
        save_name=f"{video_name}_overlay.mp4",
        mask_save_name=f"{video_name}_mask.mp4",
    )

    compress_with_motion(
        input_video=os.path.join(video_output_dir, f"{video_name}_overlay.mp4"),
        mask_video=os.path.join(video_output_dir, f"{video_name}_mask.mp4"),
        output_dir=video_output_dir
    )


if __name__ == "__main__":
    main()

2025-01-18 18:34:28,269 - INFO - Logging configurato. File di log salvato in: ../Dataset/output/test2/processing.log
2025-01-18 18:34:28,270 - INFO - Avvio della rilevazione del movimento.
2025-01-18 18:36:35,722 - INFO - Video overlay salvato in: ../Dataset/output/test2/test2/test2_overlay.mp4
2025-01-18 18:36:35,734 - INFO - Video maschera salvato in: ../Dataset/output/test2/test2/test2_mask.mp4
2025-01-18 18:36:35,745 - INFO - Avvio della compressione basata sulla maschera di movimento.
OpenCV: Couldn't read video stream from file "../Dataset/output/test2/test2_mask.mp4"
2025-01-18 18:36:35,801 - ERROR - Impossibile aprire il video maschera: ../Dataset/output/test2/test2_mask.mp4
2025-01-18 18:36:35,802 - INFO - Elaborazione completata.
